In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

    tf.summary.histogram("X", X)
    tf.summary.histogram("Weights", W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

    tf.summary.histogram("Weights", W2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

    tf.summary.histogram("Weights", W3)
    tf.summary.histogram("Model", model)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    tf.summary.scalar('cost', cost)

#########
# 신경망 모델 학습
######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Step: 1,  Cost: 1.073
Step: 2,  Cost: 1.001
Step: 3,  Cost: 0.944
Step: 4,  Cost: 0.902
Step: 5,  Cost: 0.871
Step: 6,  Cost: 0.849
Step: 7,  Cost: 0.830
Step: 8,  Cost: 0.809
Step: 9,  Cost: 0.788
Step: 10,  Cost: 0.765
Step: 11,  Cost: 0.744
Step: 12,  Cost: 0.724
Step: 13,  Cost: 0.704
Step: 14,  Cost: 0.687
Step: 15,  Cost: 0.670
Step: 16,  Cost: 0.656
Step: 17,  Cost: 0.644
Step: 18,  Cost: 0.634
Step: 19,  Cost: 0.625
Step: 20,  Cost: 0.617
Step: 21,  Cost: 0.610
Step: 22,  Cost: 0.604
Step: 23,  Cost: 0.599
Step: 24,  Cost: 0.594
Step: 25,  Cost: 0.589
Step: 26,  Cost: 0.585
Step: 27,  Cost: 0.581
Step: 28,  Cost: 0.577
Step: 29,  Cost: 0.574
Step: 30,  Cost: 0.572
Step: 31,  Cost: 0.569
Step: 32,  Cost: 0.567
Step: 33,  Cost: 0.565
Step: 34,  Cost: 0.564
Step: 35,  Cost: 0.563
Step: 36,  Cost